In [ ]:
# install simpletransformers
!pip install -U simpletransformers


# check installed version
!pip freeze | grep simpletransformers
# simpletransformers==0.28.2

Requirement already up-to-date: simpletransformers in /usr/local/lib/python3.6/dist-packages (0.49.2)
simpletransformers==0.49.2


In [ ]:
import pandas as pd
train_df = pd.read_csv('/content/train.csv',sep=";", encoding="utf8")
train_df.head()

,Unnamed: 0,Id,Impact,Urgency,IncidentType,ServiceProcessed,MailSubject,MailTextBody,ManualGroups
0,3554,INC000010587669,4-Minor/Localized,2-High,Failure,EDA_S_BA_2FA,smart card blockiert MD5:2225e4a30a5b7e592ba34...,lieber helpdesk sie haben mir den computer fr...,NaN
1,3553,INC000010585556,4-Minor/Localized,4-Low,Service Request,EDA_S_Order Management,FW: Webcam MD5:f757a42a9bdb1dfd9c8427e588586ca8,liebe kolleginnen und kollegen ich möchte für...,NaN
2,3552,INC000010585519,4-Minor/Localized,4-Low,Failure,EDA_S_Peripheriegeräte,FW: IT Support heute Nachmittag MD5:bc594d6bfd...,liebe kolleginnen und kollegen es handelt sic...,NaN
3,3549,INC000010582950,4-Minor/Localized,4-Low,Service Request,EDA_S_Benutzerunterstützung,Botschaft Warschau - Administratoren-Zugang f...,"sehr geehrte damen und herren, die botschaft ...",Benutzeranleitungen_Telefonie
4,3548,INC000010582919,4-Minor/Localized,4-Low,Service Request,EDA_S_Order Management,FW: Commande d'un écran MD5:c2b8cb3281fed4e2d4...,liebe kolleginnen und kollegen ich möchte für...,NaN


In [ ]:
test_df = pd.read_csv('/content/test_reduced.csv',sep=";", encoding="utf8")
test_df.head()

,Unnamed: 0,Id,MailSubject,MailTextBody
0,2122,INC000009681999,Traveler 0 1 2 3 austauschen (altes Modell),guten tag wir haben im 5. stock an der bg32 e...
1,202,INC000009004056,Orchestrator läuft nicht,hallo helpdesk orchestrator läuft nicht. d...
2,536,INC000009244944,Ausschalten des Bildschirms eines Laptops,liebe kolleginnen und kollegen mein kolleg...
3,22,INC000008891561,Skype,bitte den neuen mitarbeiter daniel kleis kln i...
4,2249,INC000009702828,Dell Laptop Batterie,liebes helpdesk die batterie meines dell lap...


In [ ]:
train_df.ServiceProcessed.nunique()

55

In [ ]:
train_df.ServiceProcessed.value_counts()/train_df.shape[0]

EDA_S_Order Management                  0.078735
EDA_S_BA_Mailbox                        0.075923
EDA_S_APS_OS_BasisSW                    0.073111
EDA_S_Mobile Kommunikation              0.070650
EDA_ANW_SysP eDoc                       0.060808
EDA_S_APS_PC                            0.052373
EDA_S_BA_UCC_Benutzertelefonie          0.051670
EDA_S_Netzdrucker                       0.040070
EDA_S_BA_2FA                            0.035149
EDA_S_APS_Monitor                       0.034798
EDA_ANW_Intranet/Collaboration EDA      0.031634
EDA_S_Zusätzliche Software              0.029174
EDA_S_BA_Account                        0.028120
EDA_S_Netzwerk Ausland                  0.026714
EDA_S_APS_Peripherie                    0.026714
EDA_S_Betrieb Übermitttlungssysteme     0.026362
EDA_ANW_SAP Services                    0.026362
EDA_S_Benutzerunterstützung             0.025659
EDA_S_BA_UCC_IVR                        0.024253
EDA_S_Peripheriegeräte                  0.022496
EDA_ANW_CH@World (MO

In [ ]:
print(train_df.shape)
print(test_df.shape)
print(train_df.Id.nunique())
print(test_df.Id.nunique())

(2845, 9)
(712, 4)
2845
712


In [ ]:
train_df['Text']=train_df['MailSubject']+" "+train_df['MailTextBody']
test_df['Text']=test_df['MailSubject']+" "+test_df['MailTextBody']
test_df.head(3)

,Unnamed: 0,Id,MailSubject,MailTextBody,Text
0,2122,INC000009681999,Traveler 0 1 2 3 austauschen (altes Modell),guten tag wir haben im 5. stock an der bg32 e...,Traveler 0 1 2 3 austauschen (altes Modell) gu...
1,202,INC000009004056,Orchestrator läuft nicht,hallo helpdesk orchestrator läuft nicht. d...,Orchestrator läuft nicht hallo helpdesk or...
2,536,INC000009244944,Ausschalten des Bildschirms eines Laptops,liebe kolleginnen und kollegen mein kolleg...,Ausschalten des Bildschirms eines Laptops lieb...


In [ ]:
train_df['Text']=train_df['Text'].str.replace('\\r',' ')
train_df['Text']=train_df['Text'].str.replace('\\n','')
train_df['Text']=train_df['Text'].str.replace('\r','')
train_df['Text']=train_df['Text'].str.replace('\n','')
train_df['Text']=train_df['Text'].str.replace('  ','')
train_df['Text']=train_df['Text'].str.lower()
train_df['Text']=train_df['Text'].str.replace('&nbsp;','')
train_df['Text']=train_df['Text'].str.strip()

In [ ]:
test_df['Text']=test_df['Text'].str.replace('\\r',' ')
test_df['Text']=test_df['Text'].str.replace('\\n','')
test_df['Text']=test_df['Text'].str.replace('\r','')
test_df['Text']=test_df['Text'].str.replace('\n','')
test_df['Text']=test_df['Text'].str.replace('  ','')
test_df['Text']=test_df['Text'].str.lower()
test_df['Text']=test_df['Text'].str.replace('&nbsp;','')
test_df['Text']=test_df['Text'].str.strip()

In [ ]:
train_df['Text_length']=train_df['Text'].str.len()
test_df['Text_length']=test_df['Text'].str.len()

In [ ]:
print(train_df['Text_length'].describe())
print(test_df['Text_length'].describe())

count     2841.000000
mean       623.534319
std       1427.274973
min         42.000000
25%        269.000000
50%        392.000000
75%        611.000000
max      43188.000000
Name: Text_length, dtype: float64
count     712.000000
mean      521.792135
std       481.612057
min        68.000000
25%       263.000000
50%       386.500000
75%       572.500000
max      5658.000000
Name: Text_length, dtype: float64


In [ ]:
class_list=train_df.ServiceProcessed.unique().tolist()
train_df['pred_class']=train_df.apply(lambda x:  class_list.index(x['ServiceProcessed']),axis=1)

In [ ]:
train_df1=train_df[['Text','pred_class']]
train_df1['Text']=train_df1['Text'].astype(str)
train_df1.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Text,pred_class
0,smart card blockiert md5:2225e4a30a5b7e592ba34...,0
1,fw: webcam md5:f757a42a9bdb1dfd9c8427e588586ca...,1
2,fw: it support heute nachmittag md5:bc594d6bfd...,2
3,botschaft warschau -administratoren-zugang für...,3
4,fw: commande d'un écran md5:c2b8cb3281fed4e2d4...,1


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df1 = train_test_split(train_df1, test_size=0.05)

print('train shape: ',train_df.shape)
print('test shape: ',test_df1.shape)

train shape:  (2702, 2)
test shape:  (143, 2)


In [ ]:
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 4}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "bert-base-german-cased",
    num_labels=55,
    args=train_args
)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoi

In [ ]:
# Train the model
model.train_model(train_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:377: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(1352, 1.5358681826791643)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score


test_df1['Text']=test_df1['Text'].astype(str)
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)

result

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:896: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


{'acc': 0.6433566433566433,
 'eval_loss': 1.5018022292190127,
 'f1': 0.6433566433566433,
 'mcc': 0.6267639255774063}

In [ ]:
test_df.head(2)

,Unnamed: 0,Id,MailSubject,MailTextBody,Text,Text_length
0,2122,INC000009681999,Traveler 0 1 2 3 austauschen (altes Modell),guten tag wir haben im 5. stock an der bg32 e...,traveler 0 1 2 3 austauschen (altes modell) gu...,538
1,202,INC000009004056,Orchestrator läuft nicht,hallo helpdesk orchestrator läuft nicht. d...,orchestrator läuft nicht hallo helpdesk orches...,232


In [ ]:
preds=model.predict(test_df['Text'])[0]
len(preds)

712

In [ ]:
output_df=pd.DataFrame(test_df['Id'])
output_df['Predicted']=preds
output_df.head()

,Id,Predicted
0,INC000009681999,14
1,INC000009004056,3
2,INC000009244944,10
3,INC000008891561,13
4,INC000009702828,14


In [ ]:
output_df['Predicted']=output_df['Predicted'].apply(lambda x:  class_list[x])
output_df['Predicted'].value_counts()/output_df.shape[0]

EDA_S_BA_Mailbox                       0.095506
EDA_S_Order Management                 0.084270
EDA_ANW_SysP eDoc                      0.074438
EDA_S_APS_OS_BasisSW                   0.070225
EDA_S_APS_PC                           0.063202
EDA_S_BA_UCC_Benutzertelefonie         0.063202
EDA_S_Mobile Kommunikation             0.057584
EDA_S_APS_Monitor                      0.046348
EDA_S_Netzdrucker                      0.046348
EDA_S_Zusätzliche Software             0.040730
EDA_S_BA_2FA                           0.035112
EDA_S_APS_Peripherie                   0.035112
EDA_ANW_Intranet/Collaboration EDA     0.035112
EDA_ANW_SAP Services                   0.029494
EDA_S_BA_Account                       0.029494
EDA_S_Peripheriegeräte                 0.025281
EDA_S_Netzwerk Ausland                 0.025281
EDA_S_Betrieb Übermitttlungssysteme    0.025281
EDA_S_BA_UCC_IVR                       0.022472
EDA_S_Benutzerunterstützung            0.015449
EDA_ANW_CH@World (MOSS)                0

In [ ]:
output_df.to_csv("bert_trial.csv",index=False)